# Hydra를 활용하여 hierarchical configuration 관리하기

---

## 시작하며
- hydra는 fair (Facebook Ai Research)에서 공개한 configuration 관리를 위한 오픈소스 프로젝트임.
    - omegaconf를 개발한 omry라는 사람이 크게 involve되어 있는 프로젝트.
- hydra는 동적으로 hierarchical configuration을 관리할 수 있게하는 모듈이라는 점에서 큰 의미가 있음.
- 하지만, CLI로 hydra를 사용할 경우, multi-run과 관련된 기능들이 hydra에 대한 사용이 약간 까다롭게함.
    - working directory management, multi-run with DDP ...
- 때문에 본 sample에서는 Hydra의 **hierarchical configuration** 기능만 사용할 수 있는 compose API를 중심으로 hydra를 사용할 것임.

#### Compose API는 언제 사용하는가?
- Inside a Jupyter notebook (Example)
- Inside a unit test (Example)
- In parts of your application that does not have access to the command line (Example).
- To compose multiple configuration objects (Example with Ray).

#### Compose API를 사용함으로써 hydra에서 사용하지 못하는 기능은 무엇인가?
- Tab completion (솔직히 크게 필요 없음.)
- Multirun, Working directory management (ray같은 다른 모듈에 맡기면 됨.)
- Logging management and more (직접 구현해서 사용하거나 다른 모듈에 맡기면 됨.)

> 나 같이 jupyter + vscode를 주로 사용하는 사람은 Compose API가 더 좋은 듯.

---

## Simple Configurations

```
.
├── __init__.py
├── config.yaml
├── cloud
│   └── aws.yaml
├── optimizer
│   └── adamw.yaml
├── scheduler
│   └── cosine_schedule_with_warmup.yaml
└── tokenizer
    └── autotokenizer.yaml
```

---

### Load `conf/config.yaml` considering depths

- `@hydra.main()`은 Initialize + Compose API를 통해 대체가 가능함.
- Initialization은 세가지 방식이 있으나, 여기서는 `initialize`를 사용함.

In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf, DictConfig

In [4]:
with initialize(config_path="conf"):
    configs = compose(config_name="config", overrides=[])
    print(OmegaConf.to_yaml(configs))

backbone:
  pretrained_model_name_or_path: klue/roberta-base
tokenizer:
  _target_: transformers.AutoTokenizer.from_pretrained
  pretrained_model_name_or_path: ${backbone.pretrained_model_name_or_path}
  use_fast: true



#### global config에 대한 구조 설명
- global config.yaml

```yaml
defaults:
  - _self_
  - tokenizer: autotokenizer
backbone:
  pretrained_model_name_or_path: "klue/roberta-base"
```

- defaults에 list 형태로 각 package에 대응하는 default yaml file을 지정할 수 있음.
- _self_는 special 문자로 defaults 아래에 존재하는 configuration을 추가하여 override함.
  - [_self_사용법](https://hydra.cc/docs/advanced/defaults_list/#composition-order)

- global(root)에 존재하는 config.yaml의 값을 tokenizer package에서 intepolation을 통해 사용이 가능함.

In [10]:
print(configs.tokenizer.pretrained_model_name_or_path)

klue/roberta-base


#### Instantiating objects
- _target_을 활용하여 해당 패키지에서 정의한 configuration을 활용하여 객체를 instance화 할 수 있음.
- https://hydra.cc/docs/advanced/instantiate_objects/overview/

- 장점
    - 추상화된 객체를 의도대로 잘 사용할 수 있게 캡슐화가 가능함.
- 단점
    - 추가 코드 개발이 어려움.
    - 다른 객체에 의존적인 객체에 대한 instantiation이 어려움.
        - 1.2부터는 _partial_이라는 기능을 통해 헷징하려고 함.

In [11]:
from hydra.utils import instantiate

tokenizer = instantiate(configs.tokenizer)

In [13]:
tokenizer.tokenize('토크나이저를 불러 왔어요.')

['토크', '##나이', '##저', '##를', '불러', '왔', '##어요', '.']